In [2]:
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()
for port in ports:
    print(f"Found: {port.device}")

Found: COM1
Found: COM2


In [3]:
import serial.tools.list_ports

def detect_serial_ports():
    """Detect and list all available serial ports."""
    ports = serial.tools.list_ports.comports()
    if not ports:
        print("No serial ports found.")
    else:
        print("Available serial ports:")
        for port in ports:
            print(f"Port: {port.device}, Description: {port.description}, HWID: {port.hwid}")

if __name__ == "__main__":
    detect_serial_ports()

Available serial ports:
Port: COM1, Description: Communications Port (COM1), HWID: ACPI\PNP0501\0
Port: COM2, Description: Communications Port (COM2), HWID: ACPI\PNP0501\1


In [8]:
import serial
import time

def control_xray_device(port, baudrate=9600):
    """
    Control the X-ray device by sending commands over a serial connection.

    Args:
        port (str): The serial port to which the X-ray device is connected (e.g., 'COM3').
        baudrate (int): The baud rate for the serial communication (default is 9600).
    """
    try:
        # Open the serial connection
        with serial.Serial(port, baudrate, timeout=1) as ser:
            print(f"Connected to {port} at {baudrate} baud.")

            # Turn the X-ray device ON
            ser.write(b"SETPA0\r")  # Send the SETPA0<cr> command
            print("X-ray device turned ON.")
            time.sleep(2)  # Wait for 2 seconds (or adjust as needed)

            # Turn the X-ray device OFF
            ser.write(b"RESPA0\r")  # Send the RESPA0<cr> command
            print("X-ray device turned OFF.")

    except serial.SerialException as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    # Replace 'COM3' with the correct port for your X-ray device
    control_xray_device(port="COM1")

Connected to COM1 at 9600 baud.
X-ray device turned ON.
X-ray device turned OFF.


In [ ]:
import serial
import time

# CONFIGURATION
PORT      = "COM1"     # ← change this to your port
BAUDRATE  = 9600
KV_SET    = 70         # fixed kV
MAX_UA    = 250        # your SourceBlock's max µA (e.g. 250 µA)
START_UA  = 20         # starting current (µA)
STEP_UA   = 20         # current step (µA)
END_UA    = 200        # final current (µA)
ON_TIME   = 10         # seconds X-ray ON
OFF_TIME  = 10         # seconds X-ray OFF

def to_counts(value, full_scale, counts=4095):
    return int(round(value / full_scale * counts))

def send(ser, cmd):
    ser.write(cmd.encode())
    time.sleep(0.05)

def read_counts(ser, cmd):
    """
    Send an ASCII command (with its trailing <cr>) and read back
    the integer count (0–4095) terminated by that <cr>.
    """
    ser.reset_input_buffer()
    send(ser, cmd + "\r")
    raw = ser.read_until(b'\r')
    try:
        text = raw.decode('ascii').strip()
        return int(text)
    except ValueError:
        print("⚠️ unexpected reply:", raw)
        return None

def initialize(ser):
    send(ser, "CPA1111100\r")
    send(ser, "RESPA0\r")
    send(ser, "SETPA1\r")
    time.sleep(0.1)
    send(ser, "RESPA1\r")

def set_kv(ser, kv):
    cnt = to_counts(kv, full_scale=80.0)
    send(ser, f"VA{cnt:04d}\r")
    print(f"> SET KV  : {kv} kV → VA{cnt:04d}")

def get_kv(ser):
    cnt = read_counts(ser, "RD0")
    if cnt is None: return None
    kv = cnt / 4095 * 80.0
    print(f"< MON KV : raw={cnt} → {kv:.2f} kV")
    return kv

def set_ua(ser, ua):
    cnt = to_counts(ua, full_scale=MAX_UA)
    send(ser, f"VB{cnt:04d}\r")
    print(f"> SET µA  : {ua} µA → VB{cnt:04d}")

def get_ua(ser):
    cnt = read_counts(ser, "RD1")
    if cnt is None: return None
    ua = cnt / 4095 * MAX_UA
    print(f"< MON µA : raw={cnt} → {ua:.2f} µA")
    return ua

def xray_on(ser):
    send(ser, "SETPA0\r")
    print("→ X-Ray ON")

def xray_off(ser):
    send(ser, "RESPA0\r")
    print("→ X-Ray OFF")

def pulse_sequence(ser):
    initialize(ser)

    # 1) Program fixed kV and read it back
    set_kv(ser, KV_SET)
    get_kv(ser)
    print("")

    # 2) Step current, pulse ON/OFF, and read back actual current
    for ua in range(START_UA, END_UA + 1, STEP_UA):
        set_ua(ser, ua)
        get_ua(ser)
        print("")

        xray_on(ser)
        time.sleep(ON_TIME)

        xray_off(ser)
        time.sleep(OFF_TIME)
        print("")

if __name__ == "__main__":
    try:
        with serial.Serial(PORT, BAUDRATE, timeout=1) as ser:
            print(f"Opened {PORT} @ {BAUDRATE} baud\n")
            pulse_sequence(ser)
            print("✅ Sequence complete.")
    except serial.SerialException as e:
        print("Serial error:", e)
    except Exception as e:
        print("Unexpected error:", e)


Opened COM1 @ 9600 baud

> SET KV  : 70 kV → VA3583
< MON KV : raw=10 → 0.20 kV

> SET µA  : 20 µA → VB0328
< MON µA : raw=0 → 0.00 µA

→ X-Ray ON
→ X-Ray OFF

> SET µA  : 40 µA → VB0655
< MON µA : raw=0 → 0.00 µA

→ X-Ray ON
→ X-Ray OFF

> SET µA  : 60 µA → VB0983
< MON µA : raw=0 → 0.00 µA

→ X-Ray ON
→ X-Ray OFF

> SET µA  : 80 µA → VB1310
< MON µA : raw=0 → 0.00 µA

→ X-Ray ON
→ X-Ray OFF

> SET µA  : 100 µA → VB1638
< MON µA : raw=0 → 0.00 µA

→ X-Ray ON
→ X-Ray OFF


In [1]:
import serial
import serial.tools.list_ports
import time

# CONFIGURATION
PORT      = "COM1"     # ← change this to your port
BAUDRATE  = 9600
KV_SET    = 70         # fixed kV
MAX_UA    = 250        # your SourceBlock's max µA (e.g. 250 µA)
START_UA  = 20         # starting current (µA)
STEP_UA   = 20         # current step (µA)
END_UA    = 200        # final current (µA)
ON_TIME   = 10         # seconds X-ray ON
OFF_TIME  = 10         # seconds X-ray OFF

def detect_serial_ports():
    """Detect and list all available serial ports."""
    ports = serial.tools.list_ports.comports()
    if not ports:
        print("No serial ports found.")
    else:
        print("Available serial ports:")
        for port in ports:
            print(f"Port: {port.device}, Description: {port.description}, HWID: {port.hwid}")

def to_counts(value, full_scale, counts=4095):
    return int(round(value / full_scale * counts))

def send(ser, cmd):
    ser.write(cmd.encode())
    time.sleep(0.05)

def read_counts(ser, cmd):
    """
    Send an ASCII command (with its trailing <cr>) and read back
    the integer count (0–4095) terminated by that <cr>.
    """
    ser.reset_input_buffer()
    send(ser, cmd + "\r")
    raw = ser.read_until(b'\r')
    try:
        text = raw.decode('ascii').strip()
        return int(text)
    except ValueError:
        print("⚠️ unexpected reply:", raw)
        return None

def initialize(ser):
    send(ser, "CPA1111100\r")
    send(ser, "RESPA0\r")
    send(ser, "SETPA1\r")
    time.sleep(0.1)
    send(ser, "RESPA1\r")

def set_kv(ser, kv):
    cnt = to_counts(kv, full_scale=80.0)
    send(ser, f"VA{cnt:04d}\r")
    print(f"> SET KV  : {kv} kV → VA{cnt:04d}")

def get_kv(ser):
    cnt = read_counts(ser, "RD0")
    if cnt is None: return None
    kv = cnt / 4095 * 80.0
    print(f"< MON KV : raw={cnt} → {kv:.2f} kV")
    return kv

def set_ua(ser, ua):
    cnt = to_counts(ua, full_scale=MAX_UA)
    send(ser, f"VB{cnt:04d}\r")
    print(f"> SET µA  : {ua} µA → VB{cnt:04d}")

def get_ua(ser):
    cnt = read_counts(ser, "RD1")
    if cnt is None: return None
    ua = cnt / 4095 * MAX_UA
    print(f"< MON µA : raw={cnt} → {ua:.2f} µA")
    return ua

def xray_on(ser):
    send(ser, "SETPA0\r")
    print("→ X-Ray ON")

def xray_off(ser):
    send(ser, "RESPA0\r")
    print("→ X-Ray OFF")

def pulse_sequence(ser):
    initialize(ser)

    # Ensure the X-ray is off initially
    xray_off(ser)
    time.sleep(OFF_TIME)  # Wait for the OFF_TIME duration

    # Countdown before starting the pulse sequence
    print("X-ray pulse measurement is about to start:")
    for i in range(10, 0, -1):
        print(f"Starting in {i} seconds...", end="\r")
        time.sleep(1)
    print("Starting now!                     \n")

    # 1) Program fixed kV and read it back
    set_kv(ser, KV_SET)
    get_kv(ser)
    print("")

    # 2) Step current, pulse ON/OFF, and read back actual current
    for ua in range(START_UA, END_UA + 1, STEP_UA):
        set_ua(ser, ua)
        get_ua(ser)
        print("")

        xray_on(ser)
        time.sleep(ON_TIME)

        xray_off(ser)
        time.sleep(OFF_TIME)
        print("")

if __name__ == "__main__":
    try:
        # Detect and list available serial ports
        detect_serial_ports()

        # Open the serial connection and execute the pulse sequence
        with serial.Serial(PORT, BAUDRATE, timeout=1) as ser:
            print(f"Opened {PORT} @ {BAUDRATE} baud\n")
            pulse_sequence(ser)
            print("✅ Sequence complete.")
    except serial.SerialException as e:
        print("Serial error:", e)
    except Exception as e:
        print("Unexpected error:", e)

Available serial ports:
Port: COM1, Description: Communications Port (COM1), HWID: ACPI\PNP0501\0
Port: COM2, Description: Communications Port (COM2), HWID: ACPI\PNP0501\1
Opened COM1 @ 9600 baud

→ X-Ray OFF
X-ray pulse measurement is about to start:
Starting now!                     

> SET KV  : 70 kV → VA3583
< MON KV : raw=11 → 0.21 kV

> SET µA  : 20 µA → VB0328
< MON µA : raw=0 → 0.00 µA

→ X-Ray ON
→ X-Ray OFF


KeyboardInterrupt: 

In [ ]:
import serial
import serial.tools.list_ports
import time

# CONFIGURATION
PORT      = "COM1"     # ← change this to your port
BAUDRATE  = 9600
KV_SET    = 70         # fixed kV
MAX_UA    = 250        # your SourceBlock's max µA (e.g. 250 µA)
START_UA  = 20         # starting current (µA)
STEP_UA   = 20         # current step (µA)
END_UA    = 200        # final current (µA)
ON_TIME   = 10         # seconds X-ray ON
OFF_TIME  = 10         # seconds X-ray OFF


def detect_serial_ports():
    """Detect and list all available serial ports."""
    ports = serial.tools.list_ports.comports()
    if not ports:
        print("No serial ports found.")
    else:
        print("Available serial ports:")
        for port in ports:
            print(f"Port: {port.device}, Description: {port.description}, HWID: {port.hwid}")


def to_counts(value, full_scale, counts=4095):
    return int(round(value / full_scale * counts))


def send(ser, cmd):
    ser.write(cmd.encode())
    time.sleep(0.05)


def read_counts(ser, cmd):
    """
    Send an ASCII command (with its trailing <cr>) and read back
    the integer count (0–4095) terminated by that <cr>.
    """
    ser.reset_input_buffer()
    send(ser, cmd + "\r")
    raw = ser.read_until(b'\r')
    try:
        text = raw.decode('ascii').strip()
        return int(text)
    except ValueError:
        print("⚠️ unexpected reply:", raw)
        return None


def initialize(ser):
    send(ser, "CPA1111100\r")
    send(ser, "RESPA0\r")
    send(ser, "SETPA1\r")
    time.sleep(0.1)
    send(ser, "RESPA1\r")


def set_kv(ser, kv):
    cnt = to_counts(kv, full_scale=80.0)
    send(ser, f"VA{cnt:04d}\r")
    print(f"> SET KV  : {kv} kV → VA{cnt:04d}")


def get_kv(ser):
    cnt = read_counts(ser, "RD0")
    if cnt is None:
        return None
    kv = cnt / 4095 * 80.0
    print(f"< MON KV : raw={cnt} → {kv:.2f} kV")
    return kv


def set_ua(ser, ua):
    cnt = to_counts(ua, full_scale=MAX_UA)
    send(ser, f"VB{cnt:04d}\r")
    print(f"> SET µA  : {ua} µA → VB{cnt:04d}")


def get_ua(ser):
    cnt = read_counts(ser, "RD1")
    if cnt is None:
        return None
    ua = cnt / 4095 * MAX_UA
    print(f"< MON µA : raw={cnt} → {ua:.2f} µA")
    return ua


def xray_on(ser):
    send(ser, "SETPA0\r")
    print("→ X-Ray ON")


def xray_off(ser):
    send(ser, "RESPA0\r")
    print("→ X-Ray OFF")


def pulse_sequence(ser):
    initialize(ser)

    # Ensure the X-ray is off initially
    xray_off(ser)
    time.sleep(OFF_TIME)  # Wait for the OFF_TIME duration

    # Countdown before starting the pulse sequence
    print("X-ray pulse measurement is about to start:")
    for i in range(10, 0, -1):
        print(f"Starting in {i} seconds...", end="\r")
        time.sleep(1)
    print("Starting now! \n")

    # 1) Program fixed kV and read it back
    set_kv(ser, KV_SET)
    get_kv(ser)
    print("")

    # 2) Step current, pulse ON/OFF, and read back actual current
    for ua in range(START_UA, END_UA + 1, STEP_UA):
        set_ua(ser, ua)
        get_ua(ser)
        print("")

        xray_on(ser)
        time.sleep(ON_TIME)

        xray_off(ser)
        time.sleep(OFF_TIME)
        print("")


if __name__ == "__main__":
    try:
        # Detect and list available serial ports
        detect_serial_ports()

        with serial.Serial(PORT, BAUDRATE, timeout=1) as ser:
            print(f"Opened {PORT} @ {BAUDRATE} baud\n")
            try:
                pulse_sequence(ser)
            except KeyboardInterrupt:
                print("\nInterrupted by user.")
            finally:
                # Ensure X-ray is turned off on exit for safety
                xray_off(ser)
                print("X-ray turned OFF for safety.")

            print("✅ Sequence complete.")

    except serial.SerialException as e:
        print("Serial error:", e)
    except Exception as e:
        print("Unexpected error:", e)


In [ ]:
import serial
import serial.tools.list_ports
import time

# CONFIGURATION
PORT        = "COM1"     # ← change this to your port
BAUDRATE    = 9600
KV_SET      = 70         # fixed kV
MAX_UA      = 250        # your SourceBlock's max µA (e.g. 250 µA)
START_UA    = 20         # starting current (µA)
STEP_UA     = 20         # current step (µA)
END_UA      = 200        # final current (µA)
ON_TIME     = 10         # seconds X-ray ON
OFF_TIME    = 10         # seconds X-ray OFF
SETTLE_TIME = 2          # seconds to wait after X-Ray ON before reading
WAIT_TIME   = 20         # countdown time to start pulse measurement

def detect_serial_ports():
    ports = serial.tools.list_ports.comports()
    if not ports:
        print("No serial ports found.")
    else:
        print("Available serial ports:")
        for p in ports:
            print(f"  {p.device}: {p.description}")

def to_counts(value, full_scale, counts=4095):
    """Map a physical value onto 0–counts."""
    return int(round(value / full_scale * counts))

def send(ser, cmd):
    ser.write(cmd.encode())
    time.sleep(0.05)

def read_counts(ser, cmd):
    """
    Send cmd<CR> and read back up to the next <CR>,
    parse as integer counts (0–4095).
    """
    ser.reset_input_buffer()
    send(ser, cmd + "\r")
    raw = ser.read_until(b'\r')
    text = raw.decode('ascii', errors='ignore').strip()
    try:
        return int(text)
    except ValueError:
        print("⚠️ unexpected reply from", cmd, "→", repr(raw))
        return None

def initialize(ser):
    """Clear faults per §2.3 of the DI-RS232A spec."""
    send(ser, "CPA1111100\r")
    send(ser, "RESPA0\r")
    send(ser, "SETPA1\r")
    time.sleep(0.1)
    send(ser, "RESPA1\r")

def set_kv(ser, kv):
    cnt = to_counts(kv, full_scale=80.0)
    send(ser, f"VA{cnt:04d}\r")
    print(f"> SET KV → {kv} kV (VA{cnt:04d})")

def get_kv(ser):
    cnt = read_counts(ser, "RD0")
    if cnt is None:
        return None
    kv = cnt / 4095 * 80.0
    print(f"< MON KV → {kv:.2f} kV (raw {cnt})")
    return kv

def set_ua(ser, ua):
    cnt = to_counts(ua, full_scale=MAX_UA)
    send(ser, f"VB{cnt:04d}\r")
    print(f"> SET µA → {ua} µA (VB{cnt:04d})")

def get_ua(ser):
    cnt = read_counts(ser, "RD1")
    if cnt is None:
        return None
    ua = cnt / 4095 * MAX_UA
    print(f"< MON µA → {ua:.2f} µA (raw {cnt})")
    return ua

def xray_on(ser):
    send(ser, "SETPA0\r")
    print("→ X-Ray ON")

def xray_off(ser):
    send(ser, "RESPA0\r")
    print("→ X-Ray OFF")

def pulse_sequence(ser):
    initialize(ser)
    xray_off(ser)                  # start with tube OFF
    time.sleep(OFF_TIME)

    print("X-ray pulse measurement is about to start:")
    for i in range(WAIT_TIME, 0, -1):
        print(f"Starting in {i} seconds...", end="\r")
        time.sleep(1)
    print("Starting now! \n")

    # 1) Program kV once
    set_kv(ser, KV_SET)

    # 2) Loop through current steps
    for ua in range(START_UA, END_UA + 1, STEP_UA):
        set_ua(ser, ua)

        xray_on(ser)
        # let HV & tube current ramp
        time.sleep(SETTLE_TIME)
        # now read the actual kV & µA
        get_kv(ser)
        get_ua(ser)

        # finish out the ON interval
        time.sleep(ON_TIME - SETTLE_TIME)

        xray_off(ser)
        time.sleep(OFF_TIME)
        print()

if __name__ == "__main__":
    detect_serial_ports()
    try:
        with serial.Serial(PORT, BAUDRATE, timeout=1) as ser:
            print(f"Opened {PORT} @ {BAUDRATE} baud\n")
            try:
                pulse_sequence(ser)
            except KeyboardInterrupt:
                print("\n⚠️ Interrupted by user.")
            finally:
                # guaranteed safety shut-off
                xray_off(ser)
                print("X-Ray turned OFF for safety.")
    except Exception as e:
        print("ERROR:", e)
